In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'n2o': 6.4e-07}
band = [9]
atmpro = trp
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o n2o band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = 3.2e-07
  dv = 0.001
  klin = 2.22e-20
  molecule = n2o
  ng_adju = [0, 0]
  ng_refs = [2, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(1.8, 1.66), (1.6, 1.8)]
  wgt = [(0.6, 0.5), (0.6, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.059105                    1 -0.030343
0.000750        2 -0.058551                    2 -0.030024
0.001052        3 -0.057325                    3 -0.029257
0.001476        4 -0.056075                    4 -0.028518
0.002070        5 -0.054880                    5 -0.027744
0.002904        6 -0.053691                    6 -0.026981
0.004074        7 -0.052553                    7 -0.026176
0.005714        8 -0.051491                    8 -0.025367
0.008015        9 -0.050529                    9 -0.024560
0.011243       10 -0.049723                   10 -0.023727
0.015771       11 -0.049138                   11 -0.022909
0.022122       12 -0.048866                   12 -0.022063
0.031031       13 -0.049041                   13 -0.021252
0.043528       14 -0.049419                   14 -0.020172
0.061057       15 -0.045128                   15 -0.015884
0.085645       16 -0.032382                   16 -0.006008
0.120136       17 -0.014157                   17  0.008076
0.168516       18  0.007328                   18  0.025374
0.236378       19  0.032454                   19  0.046394
0.331549       20  0.061903                   20  0.071822
0.465100       21  0.096953                   21  0.102412
0.652400       22  0.134295                   22  0.134926
0.915100       23  0.139695                   23  0.145829
1.283650       24  0.112786                   24  0.131522
1.800600       25  0.082345                   25  0.108529
2.525700       26  0.053874                   26  0.080673
3.542800       27  0.034828                   27  0.055324
4.969550       28  0.023776                   28  0.034869
6.970850       29  0.017155                   29  0.019765
9.778100       30  0.012616                   30  0.009782
13.715850      31  0.008860                   31  0.004061
19.239350      32  0.005287                   32  0.000794
26.987250      33  0.001706                   33 -0.001982
37.855300      34 -0.001978                   34 -0.005922
53.100050      35 -0.007983                   35 -0.013595
73.887500      36 -0.016113                   36 -0.024452
97.662500      37 -0.023364                   37 -0.034088
121.437500     38 -0.026910                   38 -0.039216
145.212500     39 -0.027269                   39 -0.040804
168.987500     40 -0.027805                   40 -0.041784
192.762500     41 -0.028036                   41 -0.042154
216.537500     42 -0.027897                   42 -0.041982
240.312500     43 -0.027363                   43 -0.041215
264.087500     44 -0.026481                   44 -0.039943
287.862500     45 -0.025280                   45 -0.038205
311.637500     46 -0.023787                   46 -0.036024
335.412500     47 -0.022022                   47 -0.033503
359.187500     48 -0.020007                   48 -0.030602
382.962500     49 -0.017753                   49 -0.027407
406.737500     50 -0.015271                   50 -0.023925
430.512500     51 -0.012574                   51 -0.020183
454.287500     52 -0.009668                   52 -0.016228
478.062500     53 -0.006558                   53 -0.011988
501.837500     54 -0.003240                   54 -0.007582
525.612500     55  0.000293                   55 -0.002892
549.387500     56  0.004050                   56  0.001949
573.162500     57  0.008048                   57  0.007046
596.937500     58  0.012312                   58  0.012320
620.712500     59  0.016851                   59  0.017785
644.487500     60  0.021645                   60  0.023383
668.262500     61  0.026514                   61  0.028946
692.037500     62  0.030969                   62  0.034094
715.812500     63  0.034298                   63  0.038283
739.587500     64  0.036222                   64  0.041414
763.362500     65  0.037330                   65  0.043856
78

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug      fldg      fnetg       flug          fldg   
pressure    level                                                            
0.000000    1     -22.121083  0.000000 -22.121083 -21.491254  9.026946e-08   
0.000624    2     -22.121089  0.000002 -22.121087 -21.491257  1.205811e-06   
0.000876    3     -22.121092  0.000003 -22.121089 -21.491259  1.661528e-06   
0.001229    4     -22.121095  0.000004 -22.121091 -21.491261  2.317119e-06   
0.001723    5     -22.121100  0.000006 -22.121095 -21.491263  3.258413e-06   
0.002417    6     -22.121107  0.000008 -22.121099 -21.491267  4.610702e-06   
0.003391    7     -22.121117  0.000011 -22.121105 -21.491272  6.552326e-06   
0.004757    8     -22.121130  0.000016 -22.121114 -21.491279  9.342532e-06   
0.006672    9     -22.121148  0.000023 -22.121126 -21.491289  1.335070e-05   
0.009359    10    -22.121174  0.000033 -22.121142 -21.491302  1.910591e-05   
0.013128    11    -22.121210  0.000047 -22.121164 -21.491321  2.737231e-05   
0.018415    12    -22.121261  0.000066 -22.121195 -21.491347  3.923532e-05   
0.025830    13    -22.121332  0.000095 -22.121238 -21.491384  5.626508e-05   
0.036232    14    -22.121432  0.000134 -22.121298 -21.491434  8.068517e-05   
0.050823    15    -22.121573  0.000190 -22.121383 -21.491505  1.159420e-04   
0.071291    16    -22.121768  0.000275 -22.121493 -21.491598  1.708116e-04   
0.100000    17    -22.122021  0.000419 -22.121603 -21.491713  2.650690e-04   
0.140271    18    -22.122336  0.000666 -22.121670 -21.491841  4.315924e-04   
0.196760    19    -22.122707  0.001086 -22.121621 -21.491963  7.237188e-04   
0.275997    20    -22.123105  0.001788 -22.121317 -21.492036  1.231881e-03   
0.387100    21    -22.123444  0.002942 -22.120502 -21.491966  2.107888e-03   
0.543100    22    -22.123520  0.004810 -22.118710 -21.491572  3.606259e-03   
0.761700    23    -22.122935  0.007702 -22.115233 -21.490556  6.084549e-03   
1.068500    24    -22.121585  0.011429 -22.110156 -21.488826  9.655143e-03   
1.498800    25    -22.119745  0.015337 -22.104407 -21.486556  1.408968e-02   
2.102400    26    -22.117533  0.019014 -22.098519 -21.483845  1.913923e-02   
2.949000    27    -22.115221  0.022104 -22.093117 -21.480918  2.430379e-02   
4.136600    28    -22.112939  0.024722 -22.088217 -21.477914  2.908271e-02   
5.802500    29    -22.110762  0.027237 -22.083525 -21.474943  3.299385e-02   
8.139200    30    -22.108817  0.030040 -22.078777 -21.472189  3.571135e-02   
11.417000   31    -22.107345  0.033467 -22.073878 -21.469986  3.730697e-02   
16.014700   32    -22.106797  0.037744 -22.069053 -21.468977  3.851002e-02   
22.464000   33    -22.108003  0.042990 -22.065014 -21.470409  4.054842e-02   
31.510500   34    -22.112419  0.049234 -22.063185 -21.476719  4.473433e-02   
44.200100   35    -22.122495  0.056336 -22.066158 -21.492594  5.170656e-02   
62.000000   36    -22.145346  0.062356 -22.082990 -21.529263  5.970640e-02   
85.775000   37    -22.193914  0.065546 -22.128368 -21.604278  6.585024e-02   
109.550000  38    -22.262798  0.068628 -22.194170 -21.705040  7.060151e-02   
133.325000  39    -22.348643  0.078685 -22.269958 -21.824986  8.009284e-02   
157.100000  40    -22.446097  0.099339 -22.346757 -21.958196  9.837342e-02   
180.875000  41    -22.553960  0.128895 -22.425065 -22.102757  1.252450e-01   
204.650000  42    -22.671184  0.167161 -22.504024 -22.257117  1.608731e-01   
228.425000  43    -22.796805  0.214215 -22.582589 -22.420008  2.055176e-01   
252.200000  44    -22.929795  0.270143 -22.659652 -22.590020  2.594436e-01   
275.975000  45    -23.069285  0.335053 -22.734232 -22.765956  3.228762e-01   
299.750000  46    -23.214470  0.409041 -22.805430 -22.946722  3.960319e-01   
323.525000  47    -23.364586  0.492166 -22.872420 -23.131212  4.790558e-01   
347.300000  48    -23.518903  0.584461 -22.934441 -23.318573  5.720532e-01   
371.075000  49    -23.676720  0.685932 -22.9907

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')